In [1]:
!pip install datasets --quiet
!pip install txtai --quiet
!pip install polars --quiet

In [2]:
##################################
# Data functions
##################################

import re

from datasets import load_dataset

def clean(text):
    text = text.replace("\n", " ").strip()
    return re.sub(r"\s{2,}", " ", text)


def batch(size):
    data = []
    for result in stream():
        data.append(result)
        if len(data) == size:
            yield data
            data = []

    if data:
        yield data


In [3]:
f = open("/content/Nancy.txt", "r")

In [4]:
clean = clean(f.read())

In [11]:
print(clean)

"Hi. Here are the recent amounts for the kids that I've paid: Clay:3 therapies total $645 plus $800 spending; Claire:$1200 rent and $1250 spending. Grand total is $3895 and half is $1947.50. I think it will get better soon. They are working to live within their means. Particularly Clayton but Claire is trying and doing better. Had a friendâ€™s wedding in Austin among other things. Would you send me half? Also, when you get a chance, would you let me know when you and Clay leave for Boulder and when you expect to be back? Thanks!" Clayton is on his way to you. Safe trip and Merry Christmas! Thanks. Happy Hanukkah ðŸ•Ž Ha ha. Thanks "Hi. Hope all is well. I am going to Argentina leaving the evening of 1/24 and returning on 2/6. (Hiking in Patagonia with my cousin Michelle). I want to make sure Claire has rent and other money before I head out. Kids expenses have beenâ€”Clayton$329.68 (travel), Claire $1400 (rent and other for January) and I plan to add another $1400 next week for her Feb

In [8]:
import time

from txtai import Embeddings

start = time.time()

# Create vector store. Uses SQLite + Hnswlib.
embeddings = Embeddings(
  path="sentence-transformers/all-MiniLM-L6-v2",
  content=True,
  maxlength=True
)
embeddings.index(clean)

print(f"ELAPSED = {time.time() - start:.2f}s")

ELAPSED = 6.68s


In [10]:
import polars as pl

# Search with dynamic columns and SQL
embeddings.index({"text": x, "length": len(x)} for x in clean.split("\n"))

# Search with dynamic columns and SQL
results = embeddings.search(
  """SELECT id, text, length, score
     FROM txtai
     WHERE similar('how much money')""", 1)

# Polars and Pandas DataFrame support
pl.from_dicts(results)

id,text,length,score
str,str,i64,f64
"""0""","""""Hi. Here are …",3359,0.139126
